<a href="https://colab.research.google.com/github/Itsuki-Hamano123/experiment_search_engine/blob/master/elmo_search_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import datetime
import pathlib
import os
from pprint import pprint
import pickle
import time
import numpy as np
import pandas as pd

## 形態素解析ツール用意
https://qiita.com/Fulltea/items/90f6ebe6dcceaf64eaef

In [2]:
# mecabのインストール
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.5)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.3)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already install

In [3]:
# Neologd辞書のインストール
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :     tar => ok
[install-mecab-ipadic-NEologd] :     unxz => ok
[install-mecab-ipadic-NEologd] :     xargs => ok
[install-mecab-ipadic-NEologd] :     grep => ok
[install-mecab-ipadic-NEo

In [0]:
import MeCab
import subprocess

In [5]:
!ls

drive  mecab-ipadic-neologd  sample_data


In [6]:
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
neologd_path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
# Neologd辞書の設定
m_neologd = MeCab.Tagger("-d {0}".format(neologd_path))

m_neologd_sample_txt = "8月3日に放送された「中居正広の金曜日のスマイルたちへ」(TBS系)で、1日たった5分でぽっこりおなかを解消するというダイエット方法を紹介。キンタロー。のダイエットにも密着。"

print("Mecab ipadic NEologd:\n",m_neologd.parse(m_neologd_sample_txt))

Mecab ipadic NEologd:
 8月3日	名詞,固有名詞,一般,*,*,*,8月3日,ハチガツミッカ,ハチガツミッカ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
放送	名詞,サ変接続,*,*,*,*,放送,ホウソウ,ホーソー
さ	動詞,自立,*,*,サ変・スル,未然レル接続,する,サ,サ
れ	動詞,接尾,*,*,一段,連用形,れる,レ,レ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
「	記号,括弧開,*,*,*,*,「,「,「
中居正広の金曜日のスマイルたちへ	名詞,固有名詞,一般,*,*,*,中居正広の金曜日のスマイルたちへ,ナカイマサヒロノキンヨウビノスマイルタチヘ,ナカイマサヒロノキンヨービノスマイルタチヘ
」(	記号,一般,*,*,*,*,*
TBS	名詞,固有名詞,一般,*,*,*,TBS,ティービーエス,ティービーエス
系	名詞,接尾,一般,*,*,*,系,ケイ,ケイ
)	記号,一般,*,*,*,*,*
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
、	記号,読点,*,*,*,*,、,、,、
1日	名詞,固有名詞,一般,*,*,*,1日,ツイタチ,ツイタチ
たった	副詞,助詞類接続,*,*,*,*,たった,タッタ,タッタ
5分	名詞,固有名詞,一般,*,*,*,5分,ゴフン,ゴフン
で	助詞,格助詞,一般,*,*,*,で,デ,デ
ぽっこりおなか	名詞,固有名詞,一般,*,*,*,ぽっこりおなか,ポッコリオナカ,ポッコリオナカ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
解消	名詞,サ変接続,*,*,*,*,解消,カイショウ,カイショー
する	動詞,自立,*,*,サ変・スル,基本形,する,スル,スル
という	助詞,格助詞,連語,*,*,*,という,トイウ,トユウ
ダイエット方法	名詞,固有名詞,一般,*,*,*,ダイエット方法,ダイエットホウホウ,ダイエットホウホー
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
紹介	名詞,サ変接続,*,*,*,*,紹介,ショウカイ,ショーカイ
。	記号,句点,*,*,*,*,。,。,。
キンタロー。	名詞,固有名詞,一般,*,*,*,キンタロー。,キンタロー,キンタロー
の	助詞,連体化,*,*,*,*,の,ノ,ノ
ダイエット	名詞,サ変接続,*,*,*,*,ダイエット,ダイエッ

## ELMoモデルの準備
ELMoForManyLangsを使う

In [7]:
# ELMoForManyLangsに必要なライブラリ
!pip install allennlp

### ELMoForManyLangsを利用
https://github.com/HIT-SCIR/ELMoForManyLangs#use-elmoformanylangs-programmatically

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
cd drive/My Drive/機械学習練習/全文検索エンジン作成

/content/drive/My Drive/機械学習練習/全文検索エンジン作成


In [10]:
!ls

build			   exports
configs			   faiss-gpu-1.4.0-py36_cuda9.2.148_1.tar.bz2
datasets		   info
dist			   lib
elmoformanylangs	   mecab-ipadic-neologd
elmoformanylangs.egg-info  README.md
elmo_search_engine.ipynb   setup.py
embedding


In [11]:
# ELMoForManyLangsのセットアップ
!python setup.py install

running install
running bdist_egg
running egg_info
writing elmoformanylangs.egg-info/PKG-INFO
writing dependency_links to elmoformanylangs.egg-info/dependency_links.txt
writing requirements to elmoformanylangs.egg-info/requires.txt
writing top-level names to elmoformanylangs.egg-info/top_level.txt
reading manifest file 'elmoformanylangs.egg-info/SOURCES.txt'
writing manifest file 'elmoformanylangs.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/elmoformanylangs
copying build/lib/elmoformanylangs/frontend.py -> build/bdist.linux-x86_64/egg/elmoformanylangs
copying build/lib/elmoformanylangs/__init__.py -> build/bdist.linux-x86_64/egg/elmoformanylangs
copying build/lib/elmoformanylangs/elmo.py -> build/bdist.linux-x86_64/egg/elmoformanylangs
copying build/lib/elmoformanylangs/utils.py -> build/bdist.linux-x86_64/egg/elmoformanylangs
copying build/

In [0]:
from elmoformanylangs import Embedder

In [0]:
def time_watch(func):
    """時間計測用デコレータ関数
    
    Parameters
    -----
    func : function
        実行したい関数

    """
    import functools
    import datetime
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = datetime.datetime.today()
        result = func(*args, **kwargs)
        end = datetime.datetime.today()
        print("実行時間:{time}".format(time=end - start))
        return result
    return wrapper

### 日本語の事前学習済み重みのダウンロード
embeddingディレクトに格納した<br>
https://qiita.com/mkt3/items/9577b63900109ff91665


In [0]:
# モデルのディレクトリ
MODEL_DIR = './embedding/word_unit_embedding_model/'

In [15]:
!nvidia-smi

Sat Apr 11 08:54:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [16]:
# batch_sizeは環境に合わせて変更（時間帯によってcolaboratoryのGPUの性能が変わるため）
elmo_model = Embedder(MODEL_DIR, batch_size=2)

2020-04-11 08:54:34,583 INFO: word embedding size: 387179
2020-04-11 08:54:44,102 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(387179, 100, padding_idx=3)
    )
    (projection): Linear(in_features=100, out_features=512, bias=True)
  )
  (encoder): ElmobiLm(
    (forward_layer_0): LstmCellWithProjection(
      (input_linearity): Linear(in_features=512, out_features=16384, bias=False)
      (state_linearity): Linear(in_features=512, out_features=16384, bias=True)
      (state_projection): Linear(in_features=4096, out_features=512, bias=False)
    )
    (backward_layer_0): LstmCellWithProjection(
      (input_linearity): Linear(in_features=512, out_features=16384, bias=False)
      (state_linearity): Linear(in_features=512, out_features=16384, bias=True)
      (state_projection): Linear(in_features=4096, out_features=512, bias=False)
    )
    (forward_layer_1): LstmCellWithProjection(
      (input_linearity): Line

### 入力文書の前処理を定義

In [0]:
def is_stop_word(pos):
    """
    品詞からストップワードか判断

    Parameters
    -----
    pos : string
        品詞

    Returns
    -----
    stop_word_flag : boolean
    """
    stop_word_list = ["助詞", "助動詞", "記号", "BOS/EOS"]
    stop_word_flag = True
    if pos in stop_word_list:
        stop_word_flag = False
    return stop_word_flag

def split_text(text):
    """
    文章をわかち書きする

    Parameters
    -----
    text : string

    Returns
    -----
    result : list of string
    """
    result = []
    node = m_neologd.parseToNode(text)
    while node:
        if is_stop_word(node.feature.split(",")[0]):
            result.append(node.surface)
        node = node.next
    return result

In [0]:
def preprocessing_documents(documents):
    """
    文書をELMo用に前処理

    Parameters
    -----
    documents : list of string

    Returns
    -----
    results : list of list of string
    """
    results = []
    # バグ対処
    m_neologd.parse('')
    for document in documents:
        result = split_text(document)
        results.append(result)
    return results

In [0]:
elmo_sample_docments = ["私は犬", "犬は私", "今日の天気は雪のち晴れです"]

In [20]:
proc_documents = preprocessing_documents(elmo_sample_docments)
proc_documents

[['私', '犬'], ['犬', '私'], ['今日の天気', '雪', 'のち', '晴れ']]

### ELMoで単語の分散表現を予測

In [0]:
@time_watch
def predict_word_vec(model, proc_documents):
    """
    モデルを使い単語の分散表現を予測

    Parmeters
    -----
    model : model_obj
    proc_documents : proc_documents_obj
    
    Returns
    -----
    pred_vector : list of nparray
    """
    pred_vector = model.sents2elmo(proc_documents)
    return pred_vector

In [22]:
documnets_words_vec = predict_word_vec(elmo_model, proc_documents)

2020-04-11 08:54:44,862 INFO: 2 batches, avg len: 4.7


実行時間:0:00:00.045679


In [23]:
print(documnets_words_vec[0].shape)
print(type(documnets_words_vec[0]))
documnets_words_vec

(2, 1024)
<class 'numpy.ndarray'>


[array([[-0.38220724, -0.39298365, -0.19943716, ..., -0.09475187,
          0.3819693 , -0.06013585],
        [-0.9985582 , -0.7653177 , -0.67188627, ...,  0.38484815,
         -0.14899644, -0.22106056]], dtype=float32),
 array([[-1.5832626 ,  1.3038126 , -0.06131099, ...,  0.19876009,
         -0.15746963, -0.06631211],
        [-0.92534226,  1.3555083 , -0.21629268, ..., -0.11389237,
          0.0235325 ,  0.16071147]], dtype=float32),
 array([[ 0.10117576, -0.23624992,  0.00397629, ..., -0.23287232,
          0.17794509,  0.06074806],
        [-0.90098625, -0.80577916, -0.20233391, ...,  0.13288672,
          0.47783342, -0.45927897],
        [ 1.229299  ,  0.9821803 , -0.35105655, ...,  0.05930269,
          0.5154345 , -0.4115053 ],
        [-0.6562578 ,  1.4385004 , -0.17021614, ..., -0.3254591 ,
          0.10576969,  0.1858627 ]], dtype=float32)]

#### 実行結果
「私は犬」と「犬は私」の２つの文章において、それぞれの文章で「私」「犬」は文章内での順番が考慮され、別々の分散表現が得られていることが分かる。

### 単語ベクトルから文書ベクトルを算出
fastTextの単語ベクトルの結合方法を参考に、文書内に含まれる単語ベクトルの平均値を文書ベクトルする<br>
[参考文献:https://arxiv.org/pdf/1607.01759.pdf](https://arxiv.org/pdf/1607.01759.pdf)

In [0]:
@time_watch
def calc_doc_vec(documnets_words_vec):
    """
    単語ベクトルから文書ベクトルを計算

    Parameters
    -----
    documents_word_vec : list of nparray

    Returns
    -----
    documents_vec : ndarray(文書数, 1つの文書内の次元数)

    Notes
    -----
    文書ベクトルの算出方法 : 文書内に含まれる全単語ベクトルの平均値
    """
    documnets_vec = []
    for document_words_vec in documnets_words_vec:
        document_vec = np.sum(document_words_vec, axis=0) / document_words_vec.shape[0]
        documnets_vec.append(document_vec)

    documnets_vec = np.array(documnets_vec)
    return documnets_vec

In [25]:
documnets_vec = calc_doc_vec(documnets_words_vec=documnets_words_vec)
documnets_vec

実行時間:0:00:00.000239


array([[-0.6903827 , -0.5791507 , -0.4356617 , ...,  0.14504814,
         0.11648643, -0.14059821],
       [-1.2543024 ,  1.3296604 , -0.13880183, ...,  0.04243386,
        -0.06696857,  0.04719968],
       [-0.05669233,  0.3446629 , -0.17990758, ..., -0.0915355 ,
         0.3192457 , -0.15604337]], dtype=float32)

In [26]:
print(documnets_vec.shape)
documnets_vec

(3, 1024)


array([[-0.6903827 , -0.5791507 , -0.4356617 , ...,  0.14504814,
         0.11648643, -0.14059821],
       [-1.2543024 ,  1.3296604 , -0.13880183, ...,  0.04243386,
        -0.06696857,  0.04719968],
       [-0.05669233,  0.3446629 , -0.17990758, ..., -0.0915355 ,
         0.3192457 , -0.15604337]], dtype=float32)

## faissのインストール
参考：https://qiita.com/SakuraDk/items/0c03b7ebd4aef8d6caed#colaboratory%E4%B8%8A%E3%81%A7%E3%82%A4%E3%83%B3%E3%82%B9%E3%83%88%E3%83%BC%E3%83%AB

In [0]:
# ダウンロードは一度のみ実行で大丈夫
#!wget https://anaconda.org/pytorch/faiss-gpu/1.4.0/download/linux-64/faiss-gpu-1.4.0-py36_cuda9.2.148_1.tar.bz2
#!tar xvjf faiss-gpu-1.4.0-py36_cuda9.2.148_1.tar.bz2

In [28]:
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

In [0]:
import faiss

## 検索エンジン作成

### 検索対象となるドキュメント読み込み

#### ldccデータ用の読み込み関数定義

In [0]:
def load_path_list(dir_name):
   """
   ディレクトリ配下のパス名取得
   """
   p = pathlib.Path(dir_name)
   name_list = list(p.glob("*"))
   return name_list

In [0]:
@time_watch
def load_ldcc_to_pd(dir_name):
    """
    ldccデータをDataframeに読み込み

    Parameters
    -----
    dir_name : string

    Returns
    -----
    ldcc_df : Dataframe
    """
    data_path_list = load_path_list(dir_name)
    read_message_size = 100
    
    removefile_path = pathlib.Path(DATA_DIR+'/LICENSE.txt')
    # LICENSE.txtファイルは除く
    if os.path.isfile(removefile_path):
        data_path_list.remove(removefile_path)
    print("データ件数:{size}".format(size=len(data_path_list)))

    dataset_texts = []
    for i, data_path in enumerate(data_path_list):
      with open(data_path) as f:
        # 末尾の改行コードと一部文字化けを取り除く
        data_texts = [s.rstrip().replace('\u3000','') for s in f.readlines()]
      dataset_texts.append(data_texts)
      if (i + 1) % read_message_size == 0:
        print("{i}件ファイルからテキストを読み込み完了".format(i=i+1))

    ldcc_df_columns = ['title', 'url', 'time', 'text']
    ldcc_df = pd.DataFrame(index=[], columns=ldcc_df_columns)
    for i, data_texts in enumerate(dataset_texts):
      title = data_texts[2]
      url = data_texts[0]
      time = data_texts[1]
      text = "".join(data_texts[2:])
      data_series = pd.Series([title, url, time, text], index=ldcc_df.columns)
      ldcc_df = ldcc_df.append(data_series, ignore_index=True)
      if (i + 1) % read_message_size == 0:
        print("{i}件データフレームに読み込み完了".format(i=i+1))

    ldcc_df.index = ldcc_df.index + 1
    return ldcc_df

In [32]:
DATA_DIR = "./datasets/ldcc-20140209"
file_name = "it-life-hack"

read_file_csv = "it-life-hack_20200411074951.csv"
read_bk_csv = True

if read_bk_csv:
    register_documents = pd.read_csv(DATA_DIR + '/' +  read_file_csv, index_col=0)   
else:
    register_documents = load_ldcc_to_pd(DATA_DIR + '/' + file_name)
    # csvファイルに出力
    now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    register_documents.to_csv(DATA_DIR + '/' + file_name + '_' + now + '.csv')

register_documents.head(3)

,title,url,time,text
1,戦国武将が勢揃い！戦国RPG 真・戦国バスター【iPhoneでチャンスを掴め】,http://news.livedoor.com/article/detail/6303141/,2012-02-22T17:00:00+0900,戦国武将が勢揃い！戦国RPG 真・戦国バスター【iPhoneでチャンスを掴め】こんな人におす...
2,ドコモ7.8mmの有機ELスマホ開発！スリムボディのスマートフォン「docomo NEXT ...,http://news.livedoor.com/article/detail/6295889/,2012-02-20T15:49:00+0900,ドコモ7.8mmの有機ELスマホ開発！スリムボディのスマートフォン「docomo NEXT ...
3,ネットで気軽にフォトブックの編集注文ができる！「ドリームページ」のレビューアーを30名募集,http://news.livedoor.com/article/detail/6298594/,2012-02-21T12:00:00+0900,ネットで気軽にフォトブックの編集注文ができる！「ドリームページ」のレビューアーを30名募集ソ...


### ELMoで文書ベクトルを算出

In [33]:
TEXT_COLUMN = "text"

read_file_pickle = "kaden-channel_20200409144950.pkl"
read_bk_pickle = False

if read_bk_pickle:
    with open(DATA_DIR + '/' + read_file_pickle, 'rb') as f_pkl:
       register_documents_vec = pickle.load(f_pkl)
else:
    register_documents_words_vec = predict_word_vec(elmo_model, register_documents[TEXT_COLUMN].values)
    register_documents_vec = calc_doc_vec(register_documents_words_vec)
    # pklファイルに出力
    now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    export_pkl_file = DATA_DIR + '/' + file_name + '_' + now + '.pkl'
    with open(export_pkl_file, mode='wb') as f:
        pickle.dump(register_documents_vec, f)

print("文書ベクトルの次元数確認{shape}".format(shape=register_documents_vec.shape))

2020-04-11 08:55:10,015 INFO: 435 batches, avg len: 1363.0


実行時間:0:28:35.216431
実行時間:0:00:00.812637
文書ベクトルの次元数確認(870, 1024)


#### 文書ベクトルの次元圧縮（PCA）

In [0]:
@time_watch
def train_pca_model(origin_dim, after_pca_dim, vector):
    """
    基底を探索

    Parameters
    -----
    origin_dim : int
    after_pca_dim : int
    vector : np.array(n, origin_dim)

    Returns
    -----
    pca_model : pca_obj
    """
    pca_model = faiss.PCAMatrix (origin_dim, after_pca_dim)
    pca_model.train(vector)
    assert pca_model.is_trained
    return pca_model

In [35]:
register_dim = 256

PCA_MODEL = train_pca_model(origin_dim=register_documents_vec.shape[1] , after_pca_dim=register_dim,
                            vector=register_documents_vec)

実行時間:0:00:00.388304


In [0]:
@time_watch
def dimension_reduction_pca(trained_pca_model, vector):
    """
    PCAを使い次元削減実行

    Parameters
    -----
    trained_pca_model : pca_pbj
    vector : np.array(n, origin_dim)

    Returns
    -----
    proc_pca_vector : np.array(n, after_pca_dim)
    """
    proc_pca_vector = trained_pca_model.apply_py(vector)
    return proc_pca_vector

In [37]:
register_documents__pca_vec = dimension_reduction_pca(PCA_MODEL, register_documents_vec)
print("次元削減後の次元数確認：{size}".format(size=register_documents__pca_vec.shape))

実行時間:0:00:00.010452
次元削減後の次元数確認：(870, 256)


### Indexを登録する特徴量空間の定義
検索アルゴリズムと必要な初期値はこちら参照<br>
https://github.com/facebookresearch/faiss/wiki/Faiss-indexes

In [0]:
# 登録するデータ1件分の次元数
vector_dim = register_documents__pca_vec.shape[1]
# 分割する領域の数
voronoi_num = 20
# 検索時に含める近傍の領域の数
voronoi_near = 10

#### 高速な近似アルゴリズムIndeIVFFlatを使用
https://github.com/facebookresearch/faiss/wiki/Faster-search

In [0]:
quantizer = faiss.IndexFlatL2(vector_dim)
index = faiss.IndexIVFFlat(quantizer, vector_dim, voronoi_num)

### Indexへの追加（+ボロノイ領域の学習）

In [0]:
assert not index.is_trained
index.train(register_documents__pca_vec)
assert index.is_trained

In [41]:
# idの付与
ids = np.arange(start=1, stop=register_documents__pca_vec.shape[0]+1)
print("idsの長さ{size}".format(size=len(ids)))

idsの長さ870


In [0]:
index.add_with_ids(register_documents__pca_vec, ids) 

#### 結果の保存

In [0]:
EXPORT_DIR = "./exports"
now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

In [0]:
faiss.write_index(index, EXPORT_DIR + "/features_" + now + ".index")

## 検索エンジンにクエリを投げてみる

#### 質問文の前処理

In [0]:
# 質問文
queries = ["新作のカメラに関する記事", 
           "Macの新作",
           "コンピュータ", 
           "Microsoftに関する記事", 
           "ファッション系のアプリケーション　新作",
           "人気な新商品",
           "デジタル家電　売上",
           "最新情報",
           "アマゾン　人気商品",
           "高齢者向け支援アプリ"]

In [58]:
# ELMoで単語ベクトルに変換
queries_words_vec = predict_word_vec(elmo_model, queries)
# 文書ベクトルに変換
queries_vec = calc_doc_vec(queries_words_vec)

# 次元削減
queries_pca_vec = dimension_reduction_pca(PCA_MODEL, queries_vec)
print("入力の次元{shape}".format(shape=queries_pca_vec.shape))

2020-04-11 09:31:12,142 INFO: 5 batches, avg len: 11.6


実行時間:0:00:00.196860
実行時間:0:00:00.000488
実行時間:0:00:00.000323
入力の次元(10, 256)


#### インデックスから類似文を検索する

In [0]:
@time_watch
def search_doc(queries_vec, search_doc_num, index):
    """
    類似文章の検索

    Parameters
    -----
    queries_vec : ndarray
    search_doc_num : int
        何件分まで検索してくるか
    index : index_obj
        faissのindexオブジェクト

    Returns
    -----
    D : D_obj
    I : I_obj
    """
    d, i = index.search(queries_vec, search_doc_num)
    return d, i

In [60]:
# 取得したい結果の件数を設定
search_doc_num = 10

results_distances, results_ids = search_doc(queries_pca_vec, search_doc_num, index)
print(results_ids)

実行時間:0:00:00.000199
[[773 758 808 757 424 370 139 832 309 191]
 [437 415 505 285 700  92 264 369 167  -1]
 [ 34 388 182  36 698 639  51 502 670   6]
 [415 700 437  92 505 285 369 167 264  -1]
 [148  81 325   8 464 471 764 811 717 183]
 [789 456 733 405 320 234 315 706 742 669]
 [728 626 779 361 476 131  65 636 755 511]
 [ 65 620 661 748 755 361 511 452 554 728]
 [773 758 808 424 864 608 191 309 741 832]
 [ 96 668 589  75 706 539 456 742 569  15]]


In [61]:
# 結果の描画
for i, (result_distances, result_ids) in enumerate(zip(results_distances, results_ids)):

    print("{i}番目の検索クエリ:{query}".format(i=i+1, query=queries[i]))
    print("-----検索結果-----")

    for j, (result_distance, result_id) in enumerate(zip(result_distances, result_ids)):
        if result_id != -1:
            print("第{j}位\t文書id:{id} 「{title}」 url:（{url}）"
            .format(j=j+1, id=result_id, title=register_documents["title"][result_id], url=register_documents["url"][result_id]))
            print("\t（距離{distance})\n".format(distance=result_distance))

    print("-----{i}番目の結果ここまで-----\n\n".format(i=i+1))

1番目の検索クエリ:新作のカメラに関する記事
-----検索結果-----
第1位	文書id:773 「今週末は北海道へGo！ニコニコ町会議in 北海道長万部町の詳細決定」 url:（http://news.livedoor.com/article/detail/6828627/）
	（距離13.038656234741211)

第2位	文書id:758 「町会議ラス前開催は福島県！ニコニコ町会議in 福島県三春町の詳細決定」 url:（http://news.livedoor.com/article/detail/6850409/）
	（距離13.129420280456543)

第3位	文書id:808 「いよいよラストの町会議！ニコニコ町会議in 東京都八丈島の詳細決定」 url:（http://news.livedoor.com/article/detail/6893325/）
	（距離13.154239654541016)

第4位	文書id:757 「地方の元気が日本の元気に！「ニコニコ町会議」佐賀県唐津市呼子町に1万6千人が来場」 url:（http://news.livedoor.com/article/detail/6839998/）
	（距離13.199812889099121)

第5位	文書id:424 「「写真の町」から生まれた「念力、滲透、輪郭」とは？気鋭の写真家が描く世界にドキッ」 url:（http://news.livedoor.com/article/detail/6653680/）
	（距離13.357860565185547)

第6位	文書id:370 「周辺地域の情報をゲット「ご当地ニュースbyロケタッチ新聞」リリース」 url:（http://news.livedoor.com/article/detail/6541754/）
	（距離13.705705642700195)

第7位	文書id:139 「なんという豪華キャスティング！ニコニコ超会議スーパーステージの出演者を発表！」 url:（http://news.livedoor.com/article/detail/6372414/）
	（距離13.874917030334473)

第8位	文書id:832 「町会議が震災復興を応援！福島県三春町の町会議レポー